In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn import metrics
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE, ADASYN

SyntaxError: invalid syntax (base.py, line 21)

In [2]:
#reading data
data = pd.read_csv('EOCC_New_data_with_zero_updated_columns.csv')

In [3]:
#pca
pca = PCA(n_components=1)
pc_w_m = principalComponents = pca.fit_transform(data[['Mileage_since_last _failure','weeks_in_service']])
data['PCA']=pc_w_m
#data = data.drop(['Mileage_since_last _failure','weeks_in_service'],axis = 1)

In [4]:
data.head()

,key,CAR NUMBER,LOCATION,CONDITION CODE,is_new,RESPONSE CODE,WHY MADE,PART APPLIED DATE,UPDATED PART REPAIR DATE,EOCC_POCKET,...,CN,CPRS,CSXT,FXE,KCS,KCSM,NSS,ST,UP,PCA
0,83500A,83500,A,1,1,0,0,3/23/2018,11/15/2018,3,...,3,0,3,0,0,0,4,0,15,-342312.739995
1,83500B,83500,B,1,1,0,0,6/6/2018,11/15/2018,3,...,3,0,3,0,0,0,4,0,15,-347998.750075
2,83503A,83503,A,3,0,0,0,12/19/2014,11/15/2018,3,...,6,0,6,0,0,0,5,0,23,-301619.500833
3,83503B,83503,B,3,0,0,0,10/20/2016,11/15/2018,3,...,6,0,6,0,0,0,5,0,23,-340455.610220
4,83505A,83505,A,1,1,0,0,6/22/2017,11/15/2018,3,...,9,3,12,0,1,0,2,0,28,-341196.671702


In [5]:
data['CAR_CLASS'] = pd.factorize(data.CAR_CLASS)[0]
data['EQUIP_FEAT_DESC'] = pd.factorize(data.EQUIP_FEAT_DESC)[0]
data['car_length'] = pd.factorize(data.car_length)[0]
data['car_tonnage'] = pd.factorize(data.car_tonnage)[0]
data['BUILDER_NAME'] = pd.factorize(data.BUILDER_NAME)[0]

In [ ]:
best_vals_reg=[]
for j in range(15):
    data1 = pd.DataFrame(data['CAR NUMBER'].unique())
    data1.columns=['CAR NUMBER']
    data2=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    data2.columns=['RandomNum']
    data1['RandomNum']=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    data_w_rand=pd.merge(data,data1,on='CAR NUMBER',how='left')

    data_new =data_w_rand.drop(['CAR NUMBER','key','LOCATION','RESPONSE CODE','WHY MADE','PART APPLIED DATE','UPDATED PART REPAIR DATE','UPDATED PART REPAIR DATE','BUILT DATE'],axis =1)

    #train and test  
    data_train = data_new[data_new['RandomNum'] <= 0.75]
    data_test = data_new[data_new['RandomNum'] > 0.75]

    y_train = data_train['TARGET']
    x_train = data_train.drop(['TARGET'],axis =1)
    y_test = data_test['TARGET']
    x_test = data_test.drop(['TARGET'],axis =1)
    
    ada = ADASYN(random_state=42)
    x_train_sample, y_train_sample = ada.fit_resample(x_train, y_train)

    clf = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=5000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20,                             
                                    n_jobs = -1).fit(x_train,y_train)
    vals=[]
    for i in np.linspace(0.05,0.4,50):
        predicted_probas = clf.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(x_train_sample, y_train_sample)
        vals.append(metrics.auc(fpr, tpr))
    print(max(vals))
    best_vals_reg.append(max(vals)) 

0.657271772609
0.660986647781
0.623003753897
0.625043861767
0.627154077702
0.612848765207
0.656883734761
